### Imports

In [1]:
from random import random
from tabulate import tabulate
from time import time
from math import log, e, sqrt, exp, sin, pi

### Funciones generales

In [2]:
def estimar_esperanza(N, generador):
    """
    Estima la esperanza usando la ley fuerte de los grandes numeros
    Args:
        N:            Cantidad total de valores a promediar
        generador:    Generador de la variable aleatoria de interes
    """
    esperanza = 0
    for _ in range(N):
        esperanza += generador()
    return esperanza / N

def media_muestral(datos):
    """
    Calcula la media muestral de los datos dados
    """
    n = len(datos)
    media = sum(datos) / n
    return media

def var_muestral(datos):
    """
    Calcula la varianza muestral de los datos dados
    """
    n = len(datos)
    media = media_muestral(datos)
    varianza = sum(list(map(lambda x: (x - media)**2, datos)))
    varianza = varianza / (n - 1)
    return varianza

def generar_muestra(generador, n):
    """
    Genera una muestra de tamaño n de una variable aleatoria
    Args:
        generador:    Generador de la variable aleatoria
        n        :    Tamaño de la muestra
    """
    datos = []
    for _ in range(n):
        datos.append(generador())
    return datos
    
    
def media_muestral_d(d, generador, n=100):
    """
    Estima el valor esperado de la variable aleatoria simulada por generador
    usando el estimador insesgado media muestral.
    Args:
        n:       : Tamaño minimo de la muestra
        d        : Cota para la desviacion aceptable del estimador
        generador: Generador de la variable aleatoria 
    Return:
        media    : Estimacion del valor esperado
        scuad    : Varianza muestral obtenida
        i        : Tamaño de la muestra final
    """
    media = generador()
    scuad = 0
    i = 1 
    while i <= n or sqrt(scuad/i) > d:
        i += 1
        X  = generador()
        media_previa = media
        media = media_previa + (X - media_previa) / i
        scuad = scuad * (1 - 1/(i-1)) + i*(media - media_previa) ** 2
    return media, scuad, i

def estimar_proporcion_d(d, generador, n=100):
    """
    Estima la proporción de exito de una bernoulli, con un desvio 
    de estimador acotado
    Args:
        n:       : Tamaño minimo de la muestra
        d        : Cota para la desviacion aceptable del estimador
        generador: Generador de una bernoulli 
    Return:
        p        : Estimacion de la proporcion
        var_m    : Varianza muestral obtenida
        i        : Tamaño de la muestra final
    """
    p = 0
    i = 0
    while i <= n or sqrt(p * (1 - p) / i) > d:
        i += 1
        X = generador()
        p = p + (X - p) / i
    var_m = p * (1 - p)
    return p, var_m, i

### Ejercicio 1

In [3]:
def normal_ross(mu=0, sigma=1):
    """
    Genera una variable normal estandar (por defecto) usando el algoritmo descrito 
    en el libro de Sheldon Ross, el cual esta basado en el metodo de aceptacion y 
    rechazo
    Args:
        mu   : Esperanza de la variable normal
        sigma: Desvio estandar de la variable normal 
    """
    while True:
        Y1 = -log(random()) # Exponencial de parametro 1
        Y2 = -log(random()) # Exponencial de parametro 1
        if Y2 >= (Y1 - 1) ** 2 / 2:
            # En este punto Y1 es una variable aleatoria con distribucion |Z|
            # La siguiente comparacion es el metodo para generar Z con composicion
            # a partir del generador de |Z|
            if random() < 0.5:
                return Y1 * sigma + mu
            else:
                return -Y1 * sigma + mu

In [4]:
# Tamaño esperado de la muestra 
# sigma / 0.1 < sqrt(n) <=> (sigma*10)**2 < n pero sabemos que 
# el desvio de una normal estandar es 1 por lo tanto n = 101
tamaño_muestra_esperado  = 101

# Simulamos las condiciones pedidas en el enunciado
n = 30
d = 0.1

simulacion_1 = media_muestral_d(d, normal_ross, n)
tamaño_muestra_obtenido  = simulacion_1[2]
media_muestral_1    = round(simulacion_1[0], 4)
varianza_muestral_1 = round(simulacion_1[1], 4)

datos_1 = [[tamaño_muestra_esperado, tamaño_muestra_obtenido,
            media_muestral_1, varianza_muestral_1]]

headers_1 = ["Tamaño esperado", "Tamaño obtenido", "Media muestral", 
             "Varianza muestral"]

print(tabulate(datos_1, headers=headers_1, tablefmt="pretty"))

+-----------------+-----------------+----------------+-------------------+
| Tamaño esperado | Tamaño obtenido | Media muestral | Varianza muestral |
+-----------------+-----------------+----------------+-------------------+
|       101       |       97        |     0.0376     |      0.9684       |
+-----------------+-----------------+----------------+-------------------+


### Ejercicio 2

In [7]:
def generar_i():
    """
    Devuelve un valor aleatorio de la funcion g determinada por el metodo de
    Monte Carlo para el inciso i
    """
    U = random()
    return exp(U) / sqrt(2*U)

def generar_ii():
    """
    Devuelve un valor aleatorio de la funcion g determinada por el metodo de
    Monte Carlo para el inciso ii
    """
    U = random()
    return 1/U**2 * (1/U - 1)**2 * exp(-(1/U - 1)**2)

In [8]:
# Estimacion de las integrales por Monte Carlo
d = 0.01
estimacion_i  = media_muestral_d(d, generar_i)
estimacion_ii = media_muestral_d(d, generar_ii)

datos_2   = [["i", estimacion_i[0], estimacion_i[2]],
             ["ii", estimacion_ii[0]*2, estimacion_ii[2]]
            ]
headers_2 = ["Inciso", "Estimacion", "Datos generados"]

print(tabulate(datos_2, headers=headers_2, tablefmt="pretty"))

+--------+--------------------+-----------------+
| Inciso |     Estimacion     | Datos generados |
+--------+--------------------+-----------------+
|   i    | 2.085988307858936  |      44145      |
|   ii   | 0.8868807748078458 |      3185       |
+--------+--------------------+-----------------+


### Ejercicio 3

In [5]:
def integral_i():
    """
    Una funcion que sirve para estimar la integral del inciso i usando el
    metodo de Monte Carlo
    """
    U = random()
    return sin((U+1)*pi) / (U+1)

def integral_ii():
    """
    Una funcion que sirve para estimar la integral del inciso ii usando el
    metodo de Monte Carlo
    """
    U = random()
    return 1/U**2 * (3 / (3 + (1/U - 1)**4))

In [6]:
# Parametros para estimar las integrales con un intervalo de confianza del 95%
# con el SEMIANCHO no mayor a 0.001 del intervalo
z_alpha_2 = 1.96
d = 0.001 / z_alpha_2  

# Estimamos las integrales simulando hasta satisfacer las condiciones dadas en 
# el enunciado (un intervalo no mayor al 0.001 con una confianza del 95%)
b_3_i  = media_muestral_d(d, integral_i)
b_3_ii = media_muestral_d(d, integral_ii)


# Generamos las muestras de los distintos tamaños pedidos en la tabla
numero_sim = [1000, 5000, 7000]
sim_3_i    = list(map(lambda n: generar_muestra(integral_i, n), numero_sim))  
sim_3_ii   = list(map(lambda n: generar_muestra(integral_ii, n), numero_sim))

# Calculamos los estimadores puntuales en las muestras obtenidas
sim_3_i  = list(map(lambda datos: [media_muestral(datos), 
                                   var_muestral(datos), len(datos)],
                    sim_3_i))

sim_3_ii = list(map(lambda datos: [media_muestral(datos),
                                   var_muestral(datos), len(datos)], 
                    sim_3_ii))

# Agregamos las simulaciones anteriores a las listas correspondientes
sim_3_i.append(b_3_i)
sim_3_ii.append(b_3_ii)

# Funcion para calcular el intervalo obtenido en las simulaciones 
# a partir de los resultados en la simulacion
intervalo = lambda media, var, n:  [round(media - sqrt(var/n)* z_alpha_2, 4), 
                                    round(media + sqrt(var/n)* z_alpha_2, 4)]

# Formateo de datos
datos_3_i  = list(map(lambda x: [x[2], intervalo(x[0], x[1], x[2]), round(x[0], 4)], sim_3_i))
datos_3_ii = list(map(lambda x: [x[2], intervalo(x[0], x[1], x[2]), round(x[0], 4)], sim_3_ii))
headers_3  = ["N° de simulaciones", "Intervalo", "Valor estimado"]

print("\nInciso i")
print(tabulate(datos_3_i, headers=headers_3, tablefmt="pretty"))
print("\nInciso ii")
print(tabulate(datos_3_ii, headers=headers_3, tablefmt="pretty"))


Inciso i
+--------------------+--------------------+----------------+
| N° de simulaciones |     Intervalo      | Valor estimado |
+--------------------+--------------------+----------------+
|        1000        | [-0.4456, -0.4196] |    -0.4326     |
|        5000        | [-0.4456, -0.434]  |    -0.4398     |
|        7000        | [-0.4388, -0.429]  |    -0.4339     |
|       170114       | [-0.4351, -0.4331] |    -0.4341     |
+--------------------+--------------------+----------------+

Inciso ii
+--------------------+------------------+----------------+
| N° de simulaciones |    Intervalo     | Valor estimado |
+--------------------+------------------+----------------+
|        1000        | [1.3746, 1.4952] |     1.4349     |
|        5000        | [1.4421, 1.4966] |     1.4693     |
|        7000        | [1.4343, 1.4799] |     1.4571     |
|      3660624       | [1.4606, 1.4626] |     1.4616     |
+--------------------+------------------+----------------+


### Ejercicio 4

In [11]:
def generar_N():
    """
    Genera la variable aleatoria N del ejercicio 4
    """
    n = 0
    suma = 0
    while suma < 1:
        n += 1
        suma += random()
    return n

def estimador_e(n):
    """
    Genera valores aleatorios del estimador de e (media muestral)
    Args:
        n: Tamaño de la muestra
    """
    muestra      = generar_muestra(generar_N, n)
    estimacion_e = media_muestral(muestra)
    return estimacion_e

In [12]:
# Inciso c
# Calculo la varianza muestral del estimador e
n = 1000
muestra_estimador  = generar_muestra(lambda: estimador_e(n), n)
varianza_estimador = var_muestral(muestra_estimador) 

# Estimamos e por intervalos de confianza al 95% con un ancho de 0.1
z_alpha_2 = 1.96
L = 0.1
d = 0.1 / (2 * z_alpha_2)
estimacion_e = media_muestral_d(d, generar_N)[0]


datos_4   = [[e, estimacion_e, varianza_estimador]]
headers_4 = ["Valor real e","Estimacion de e", "Varianza estimador"]
print(tabulate(datos_4, headers=headers_4, tablefmt="pretty"))

+-------------------+--------------------+-----------------------+
|   Valor real e    |  Estimacion de e   |  Varianza estimador   |
+-------------------+--------------------+-----------------------+
| 2.718281828459045 | 2.6688311688311654 | 0.0007131215175175175 |
+-------------------+--------------------+-----------------------+


### Ejercicio 5

In [13]:
def generar_M():
    """
    Genera la variable aleatoria M dada en el ejercicio 5
    """
    numero_actual = random()
    n = 2
    while True:
        numero_proximo = random()
        if numero_actual < numero_proximo:
            numero_actual = numero_proximo
            n+=1
        else:
            return n
        
def estimador_e(n):
    """
    Genera valores aleatorios del estimador de e (media muestral)
    Args:
        n: Tamaño de la muestra
    """
    muestra      = generar_muestra(generar_M, n)
    estimacion_e = media_muestral(muestra)
    return estimacion_e

In [14]:
# Estimamos el valor de e con 1000 simulaciones
n = 1000

# Inciso c
muestra_m    = generar_muestra(generar_M, n)
estimacion_e = media_muestral(muestra_m)

# Inciso d 
# Varianza del estimador de e (media muestral de M)
muestra_estimador      = generar_muestra(lambda: estimador_e(n), n)
var_muestral_estimador = var_muestral(muestra_estimador) 

# Estimacion de e con intervalos de confianza del 95% y largo del 0.01
z_alpha_2 = 1.96
L = 0.01
d = L / (2 * z_alpha_2)
estimacion_e_confianza = media_muestral_d(d, generar_M)[0]

data_5    = [[estimacion_e, var_muestral_estimador, estimacion_e_confianza]]
headers_5 = ["Valor por estimador", "Varianza del estimador", 
             "Estimacion por intervalo"]

print(tabulate(data_5, headers=headers_5, tablefmt="pretty"))

+---------------------+------------------------+--------------------------+
| Valor por estimador | Varianza del estimador | Estimacion por intervalo |
+---------------------+------------------------+--------------------------+
|        2.681        | 0.0007396332332332345  |    2.7176945166142317    |
+---------------------+------------------------+--------------------------+


### Ejercicio 6

In [15]:
def cae_en_circulo():
    """
    Genera la variable aleatoria bernoulli que modela el hecho de que un punto
    aleatorio del cuadrado [-1,1]x[-1,1] caiga en el circulo unitario
    """
    X = random()*2 - 1    # U(-1, 1)  
    Y = random()*2 - 1    # U(-1, 1)
    return int(X**2 + Y**2 <= 1)

In [16]:
# Estimamos pi por intervalo de confianza al 95% con una cota de 0.1
L = 0.1
z_alpha_2 = 1.96         # Valor de la normal para la confianza al 95%
d = L / (8 * z_alpha_2)  # Por justificacion analitica

simular_pi = estimar_proporcion_d(d, cae_en_circulo) 

estimacion_pi  = simular_pi[0] * 4  # Porque la media muestral de cae_en_circulo estima pi/4
tamaño_muestra = simular_pi[2]

data_6    = [[pi, estimacion_pi, tamaño_muestra]]
headers_6 = ["Valor de pi", "Estimacion de pi", "Tamaño final de la muestra"]

print(tabulate(data_6, headers=headers_6, tablefmt="pretty"))

+-------------------+-------------------+----------------------------+
|    Valor de pi    | Estimacion de pi  | Tamaño final de la muestra |
+-------------------+-------------------+----------------------------+
| 3.141592653589793 | 3.129147767963719 |            4189            |
+-------------------+-------------------+----------------------------+


### Funciones generales para el metodo bootstrap

In [17]:
def generar_empirica(datos):
    """
    Genera la variable aleatoria con distribucion empirica para los
    datos dados
    """
    n = len(datos)
    U = int(random() * n)
    return datos[U]

def generar_muestra_bootstrap(datos):
    """
    Genera una muestra bootstrap a partir de la distribucion empirica
    de los datos dados
    """
    n = len(datos)        # Tamaño de la muestra bootstrap apropiado
    muestra = generar_muestra(lambda: generar_empirica(datos), n)
    return muestra

def esperanza_empirica(datos):
    """
    Calcula la esperanza empirica de los datos dados
    """
    esperanza = sum(datos) / len(datos)
    return esperanza

def varianza_empirica(datos):
    """
    Calcula la varianza empirica de los datos datos
    """
    mu = esperanza_empirica(datos) # Suponemos que no conocemos el valor esperado real
    varianza_empirica = sum(list(map(lambda x: (x-mu)**2, datos))) / len(datos)
    return varianza_empirica


def var_muestral_mu(datos, mu):
    """
    Calcula la varianza muestral sin estimar la esperanza de los datos
    Args:
        mu: Valor esperado de los datos
    """
    var = sum(list(map(lambda x: (x-mu)**2, datos))) / len(datos)
    return var

### Ejercicio 7

In [18]:
# Datos dados
datos_obtenidos_7 = [56, 101, 78, 67, 93, 87,64, 72, 80, 69]

# Esperanza por distribucion empirica 
esperanza_empirica_7 = esperanza_empirica(datos_obtenidos_7)

# Generamos la bernoulli que modela el hecho de que la replicacion bootstrap
# pertenezca al intervalo (-5, 5) dado en el ejercicio
def bernoulli_7():
    """
    Genera la variable bernoulli con probabilidad de exito
    igual a p(-5 < replicacion < 5)
    """
    estimador = lambda x: media_muestral(x) - esperanza_empirica_7
    muestra_bootstrap     = generar_muestra_bootstrap(datos_obtenidos_7)
    replicacion_bootstrap = estimador(muestra_bootstrap)
    return -5 < replicacion_bootstrap < 5 

In [19]:
# Estimamos la probabilidad p con N muestras bootstraps
N = 100000
p_estimado = estimar_esperanza(N, bernoulli_7)
print("p(-5 < X̄-μ < 5) ~", p_estimado)

p(-5 < X̄-μ < 5) ~ 0.76047


### Ejercicio 9

In [20]:
datos_obtenidos_9 = [3.0592, 2.3304, 2.8548, 1.2546, 2.1628, 
                     4.9828, 5.4259, 0.9078, 4.5811, 3.2749]

# Esperanza empirica obtenida de los datos dados
esperanza_empirica_9 = esperanza_empirica(datos_obtenidos_9)

# Generamos la bernoulli que modela el hecho de que la replicacion bootstrap
# de nuestro estimador T sea mayor a a = 2.5
def bernoulli_9(a):
    """
    Genera la variable bernoulli con probabilidad de exito
    igual a p(replicacion > a)
    """
    mu = esperanza_empirica_9
    T  = lambda x: (media_muestral(x) - mu) / sqrt(var_muestral_mu(x, mu)/len(x))
    muestra_b = generar_muestra_bootstrap(datos_obtenidos_9)
    replicacion = T(muestra_b)
    return int(replicacion > a) 

In [21]:
# inciso b
media_muestral_9 = media_muestral(datos_obtenidos_9)
var_muestral_9   = var_muestral(datos_obtenidos_9)

# inciso c
a = 2.50
N = 100000    # Cantidad de simulaciones
p_estimado = round(estimar_esperanza(N, lambda: bernoulli_9(a)), 5)

datos_9   = [[media_muestral_9, var_muestral_9, p_estimado]]
headers_9 = ["Media muestral", "Varianza muestral", "P(T > 2.5)"]
print(tabulate(datos_9, headers=headers_9, tablefmt="pretty"))

+--------------------+-------------------+------------+
|   Media muestral   | Varianza muestral | P(T > 2.5) |
+--------------------+-------------------+------------+
| 3.0834300000000003 |    2.324433989    |  0.00158   |
+--------------------+-------------------+------------+


### Ejercicio 10

In [22]:
datos_obtenidos_10 = [7.5, 12.3, 8.8, 7.9, 9.3, 10.4, 10.9, 9.6, 9.1, 11.2]

# Esperanza empirica obtenida de los datos dados
esperanza_empirica_10 = esperanza_empirica(datos_obtenidos_10)

def replicacion_bootstrap_10():
    """
    Genera una muestra bootstrap y luego le calcula su replicacion para estimar
    el ECM
    """
    muestra_b   = generar_muestra_bootstrap(datos_obtenidos_10)
    replicacion = (media_muestral(muestra_b) - esperanza_empirica_10) ** 2
    return replicacion

def bernoulli_10():
    """
    Genera la bernoulli con probabilidad de exito 
    igual a la p(8 <= replicacion <= 10)
    """
    muestra_b = generar_muestra_bootstrap(datos_obtenidos_10)
    estimador = media_muestral
    replicacion = estimador(muestra_b)
    return int(8 <= replicacion <= 10)

In [23]:
# Estimamos ECM(X̄, μ)
N = 100000
ECM_10 = round(estimar_esperanza(N, replicacion_bootstrap_10), 5)

# Estimamos la probabilidad de que X̄ este en el intervalo [8, 10]
p_estimado_10 = estimar_esperanza(N, bernoulli_10)

datos_10   = [[esperanza_empirica_10, ECM_10, p_estimado_10]]
headers_10 = ["Esperanza empirica", "ECM estimado", "P(8<=X̄<=10) estimada"]

print(tabulate(datos_10, headers=headers_10, tablefmt="pretty"))

+--------------------+--------------+----------------------+
| Esperanza empirica | ECM estimado | P(8<=X̄<=10) estimada |
+--------------------+--------------+----------------------+
|        9.7         |   0.20356    |        0.7516        |
+--------------------+--------------+----------------------+


### Ejercicio 11

In [24]:
datos_obtenidos_11 = [144.98, 145.04, 145.02, 145.04, 145.03, 
                      145.03, 145.04, 144.97, 145.05, 145.03, 
                      145.02, 145,    145.02]

# Estimamos los parametros necesarios para nuestro estimador
e_empirica_11 = esperanza_empirica(datos_obtenidos_11) 
v_empirica_11 = varianza_empirica(datos_obtenidos_11)

# Generamos la bernoulli que modela el hecho de que la replicacion bootstrap
# de nuestro estimador |s² - σ²| sea mayor a 0.02
def bernoulli_11():
    """
    Genera una variable aleatoria con probabilidad de exito
    igual a p(|s² - σ²| > 0.02)
    """
    sigma2 = v_empirica_11     # Estimacion de la varianza
    mu     = e_empirica_11     # Estimacion de la esperanza
    
    muestra_b   = generar_muestra_bootstrap(datos_obtenidos_11) 
    estimador   = lambda x: abs(var_muestral_mu(x, mu) - sigma2)
    replicacion = estimador(muestra_b)
    
    return int(replicacion > 0.02)

In [25]:
# Estimamos la probabilidad p = p(|s² - σ²| > 0.02) usando Monte Carlo
N = 10000  # Cantidad de muestras bootstrap
p_estimado_11 = estimar_esperanza(N, bernoulli_11)

datos_11   = [[e_empirica_11, v_empirica_11, p_estimado_11]]
headers_11 = ["Esperanza empirica", "Varianza empirica", "P(|s² - σ²| > 0.02) estimado"]

print(tabulate(datos_11, headers=headers_11, tablefmt="pretty"))

+--------------------+-----------------------+------------------------------+
| Esperanza empirica |   Varianza empirica   | P(|s² - σ²| > 0.02) estimado |
+--------------------+-----------------------+------------------------------+
| 145.02076923076922 | 0.0005301775147929542 |             0.0              |
+--------------------+-----------------------+------------------------------+


In [40]:
def generar_exp(Lambda):
    """
    Genera una variable aleatoria exponencial usando el metodo de la transformada
    inversa
    Args:
        Lambda: Parametro de la variable aleatoria exponencial
    """
    U = random()
    return -log(U) / Lambda


def eventos_poisson(Lambda, T):
    """
    Genera la variable aleatoria N(T) ligada a un proceso de poisson homogeneo
    de parametro Lambda usando el hecho de que el tiempo entre arribos es una 
    variable exponencial de parametro Lambda
    Args:
        Lambda: Intensidad del proceso de poisson
        T     : Limite del intervalo [0, T]
    Return:
        N_T    : Cantidad de eventos ocurridos en el intervalo [0, T] 
        eventos: Momento en que ocurrio cada evento 
    """
    t, N_T, eventos = 0, 0, []
    while t < T:
        U  = 1 - random()
        t += - log(U) / Lambda # Exponencial de parametro lambda
        if t <= T:
            N_T += 1
            eventos.append(t)
    return N_T, eventos


def simular_servidores(tu, Lambda, t):
    """
    Simula el tiempo promedio de uso del servicio de los clientes efectivos 
    en un dia 
    Args:
        tu                     : Tiempo medio de uso del servicio
        Lambda                 : Frecuencia con la que llegan los clientes
        t                      : Cota del intervalo de tiempo [0, t]
    Return:
        t_uso_servicio         : Tiempo promedio de uso promedio de los clientes efectivos
        clientes_atendidos     : Cantidad de clientes efectivos
        cantidad_clientes_p    : Cantidad de clientes potenciales
    """
    simular_servicio     = lambda : generar_exp(tu)        # Simulacion del tiempo de uso del servicio
    servidores_en_uso    = []                              # [] indica todos los servidores libres
    potenciales_clientes = eventos_poisson(Lambda, t)[1]   # Momentos en que llegan los clientes
    cantidad_clientes_p  = len(potenciales_clientes)  
    t_uso_servicio       = 0                               # Tiempo de uso total del servicio
    clientes_atendidos   = 0
    # Atendemos los clientes
    while potenciales_clientes != []:
        momento_llegada = potenciales_clientes[0]
        # Actualizo el tiempo en los servidores
        servidores_en_uso = list(filter(lambda x: x > momento_llegada, servidores_en_uso))
        # Veo si hay un servidor disponible
        if len(servidores_en_uso) <= 3:
            tiempo_servicio = simular_servicio()
            momento_salida  = tiempo_servicio + momento_llegada
            servidores_en_uso.append(momento_salida)
            t_uso_servicio += tiempo_servicio
            clientes_atendidos += 1
            
        # Desencolamos al cliente
        potenciales_clientes = potenciales_clientes[1:]
    t_uso_servicio = t_uso_servicio / cantidad_clientes_p
    return t_uso_servicio, clientes_atendidos, cantidad_clientes_p

def replicacion_bootstrap_12():
    """
    Genera una muestra bootstrap y luego le calcula su replicacion para estimar
    el ECM 
    """
    muestra_b   = generar_muestra_bootstrap(datos_obtenidos_12)
    replicacion = (media_muestral(muestra_b) - e_empirica_12) ** 2
    return replicacion

#### Aplicacion del metodo bootstrap

In [50]:
# Parametros
t_uso  = 4.2
Lambda = 4
t      = 8

# Primero obtenemos 10 datos de la simulacion
datos_obtenidos_12 = generar_muestra(lambda: simular_servidores(t_uso, Lambda, t)[0], 15)

# Calculo la esperanza empirica para estimar el ECM del estimador
e_empirica_12 = esperanza_empirica(datos_obtenidos_12)

# Estimamos ECM con N muestras bootstrap
N = 10000
ECM_12 = estimar_esperanza(N, replicacion_bootstrap_12)

datos_12 = [[e_empirica_12, ECM_12]]
headers_12 = ["Horas promedio de servicio", "ECM estimada del estimador"]

print(tabulate(datos_12, headers=headers_12, tablefmt="pretty"))

+----------------------------+----------------------------+
| Horas promedio de servicio | ECM estimada del estimador |
+----------------------------+----------------------------+
|    0.23884282661213888     |   0.00010129315028953236   |
+----------------------------+----------------------------+
